In [1]:
import sqlite3
import datetime
conn = sqlite3.connect('/home/arcyleung/Projects/frigate/config/frigate.db')
cursor = conn.cursor()

print("Tables:")
for row in cursor.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    print(row[0])

# Define the start time (now - 24 hours)
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(hours=24)

# print("schema:", cursor.execute("SELECT * FROM event").description)

# Read the "event" table and filter records with "start_time" within the last 24 hours
print("start_time:", start_time.timestamp())
print("end_time:", end_time.timestamp())
cursor.execute("SELECT * FROM event WHERE start_time >= :start_time AND start_time <= :end_time", {'start_time': start_time.timestamp(), 'end_time': end_time.timestamp()})
events = cursor.fetchall()

# Print the filtered events
print(len(events), "events found")
ids = [ f"{event[2]}-{event[0]}" for event in events ]
# for event in events:
#     print(event)
#     print(event[0], event[1])
print(ids)

conn.close()


Tables:
migratehistory
recordings
timeline
event
regions
transcribed
start_time: 1726269802.724936
end_time: 1726356202.724936
151 events found
['front-1726356190.982318-7t2mrp', 'front-1726356190.565374-tn79um', 'front-1726355933.029623-yc41a1', 'front-1726355914.411895-l9r499', 'front-1726354992.453692-3j7ka0', 'front-1726354902.670007-4y1xjh', 'front-1726354743.590189-ibzczy', 'front-1726354506.285611-h2r2zu', 'front-1726354483.738028-r441gs', 'front-1726354324.178057-p3mhcw', 'front-1726353858.895057-octnpe', 'front-1726353567.689386-cvdc7r', 'front-1726353566.90992-hrh2p2', 'front-1726353559.05066-nbiv7f', 'front-1726352694.693961-amdj0g', 'front-1726352682.412965-ug041w', 'front-1726351097.954426-i30jnh', 'front-1726351093.64947-rxmck3', 'front-1726351089.759448-ktcdze', 'front-1726349770.178882-qcaput', 'front-1726348519.69014-jaj62l', 'front-1726347986.468268-h3kvxv', 'front-1726347942.42262-4iioz8', 'front-1726347114.384188-bympb4', 'front-1726347102.666271-2f4it3', 'front-172

In [2]:
from dotenv import load_dotenv
import os

def get_filenames(path, ids=None):
    filenames = []
    if ids is not None:
        for id in ids:
            filenames.append(os.path.join(path, id) + "-clean.png")
    else:
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith('-clean.png'):
                    filenames.append(os.path.join(root, file))
    return filenames

load_dotenv()
prefix = os.getenv("FRIGATE_CLIPS_PATH")

filenames = get_filenames(prefix, ids)
print(prefix, filenames)

/home/arcyleung/Projects/frigate/media/frigate/clips ['/home/arcyleung/Projects/frigate/media/frigate/clips/front-1726356190.982318-7t2mrp-clean.png', '/home/arcyleung/Projects/frigate/media/frigate/clips/front-1726356190.565374-tn79um-clean.png', '/home/arcyleung/Projects/frigate/media/frigate/clips/front-1726355933.029623-yc41a1-clean.png', '/home/arcyleung/Projects/frigate/media/frigate/clips/front-1726355914.411895-l9r499-clean.png', '/home/arcyleung/Projects/frigate/media/frigate/clips/front-1726354992.453692-3j7ka0-clean.png', '/home/arcyleung/Projects/frigate/media/frigate/clips/front-1726354902.670007-4y1xjh-clean.png', '/home/arcyleung/Projects/frigate/media/frigate/clips/front-1726354743.590189-ibzczy-clean.png', '/home/arcyleung/Projects/frigate/media/frigate/clips/front-1726354506.285611-h2r2zu-clean.png', '/home/arcyleung/Projects/frigate/media/frigate/clips/front-1726354483.738028-r441gs-clean.png', '/home/arcyleung/Projects/frigate/media/frigate/clips/front-1726354324.17

In [3]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True, load_in_4bit=True, use_flash_attention_2=False)
# model.to("cuda:0")

# Define a chat histiry and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image")
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "Describe any people in the scene, what clothing they are wearing, orientation with other objects, anything they are holding and what do they appear to be doing. Also describe prominent vehicles or legible text, and do not mention anything that shoule be ordinary."},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)


/home/arcyleung/Projects/llm-nvr/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.85s/it]


In [4]:

from PIL import Image
import datetime
import json

def transcribe_one(filename, responses):
    image = Image.open(filename)
    inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")

    # autoregressively complete prompt
    output = model.generate(**inputs, max_new_tokens=150)

    text = processor.decode(output[0], skip_special_tokens=True)
    responses[filename] = text[len(prompt) - 5:]


def upsert_transcriptions_to_sqlite3(responses, cursor=None, transcribed_at=None):
    import sqlite3
    import datetime
    conn = sqlite3.connect('/home/arcyleung/Projects/frigate/config/frigate.db')
    cursor = conn.cursor()

    if cursor is None:
        print("No db cursor provided")
        return
    # Create table if it doesn't exist
    # Must make frigate.db have permissions to write to it
    cursor.execute("CREATE TABLE IF NOT EXISTS transcribed (id TEXT PRIMARY KEY, transcript TEXT, transcribed_at INTEGER)")

    cursor.execute("BEGIN")
    if transcribed_at is None:
        transcribed_at = int(datetime.datetime.now().timestamp())

    def filepath_to_id(filename):
        print(filename)
        splitted = filename.split("-")
        return splitted[1] + "-" + splitted[2]

    records = [(filepath_to_id(filename), text, transcribed_at) for filename, text in responses.items()]
    try:
        cursor.executemany("INSERT INTO transcribed VALUES(?, ?, ?)", records);
        cursor.execute("COMMIT")
    except Exception as ex:
        cursor.execute("ROLLBACK")
        print(ex)

    conn.close()


In [5]:
# responses = json.load(open(f"/home/arcyleung/Projects/llm-nvr/transcribed/analyze_2024-08-12T20:08:12.464352.json", "r"))

# print(responses)
# upsert_transcriptions_to_sqlite3(responses, cursor)

In [5]:


responses = {}
today = datetime.datetime.now().isoformat()

for idx, filename in enumerate(filenames):
    print(f"{idx + 1} of {len(filenames)}")
    transcribe_one(filename, responses)

upsert_transcriptions_to_sqlite3(responses, cursor)

# transcribe_one("/tmp/storage/clips/front-1722009392.383259-1jcmy8-clean.png", responses)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1 of 151


/home/arcyleung/Projects/llm-nvr/venv/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2 of 151


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

61 of 151


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


62 of 151


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


63 of 151


In [7]:

print(responses)

json.dump(responses, open(f"transcribed/analyze_{today}.json", "w"))


{'/home/arcyleung/Projects/frigate/media/frigate/clips/front-1723687423.885892-jq1jea-clean.png': 'In the image, there is a person walking on the sidewalk. The individual is wearing a dark-colored top and dark pants, and they appear to be facing away from the camera, walking towards the right side of the image. The person is holding what seems to be a bag or a small item in their hand.\n\nProminent vehicles in the scene include a car with its headlights on, which is driving towards the person walking. There is also a truck visible in the background, and a motorcycle parked on the side of the street.\n\nThe image is taken at night, as indicated by the darkness of the sky and the illuminated streetlights. The streetlights are green, and there are other lights visible in the', '/home/arcyleung/Projects/frigate/media/frigate/clips/front-1723684336.274846-0lkcg8-clean.png': 'In the image, there are no people visible. The scene is dominated by vehicles, with a car prominently featured in the